In [47]:
#Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error

In [11]:
# Load dataset
data = pd.read_csv('/content/heart_disease_dataset_UCI - heart_disease_dataset_UCI.csv')
X = data.drop(columns=['target'])  # Assuming 'target' is the column to predict
y = data['target']

In [12]:
from sklearn.preprocessing import StandardScaler

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

#Task 01

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, random_state = 2)

In [17]:
linear_model = LinearRegression()
linear_model.fit(X_train, Y_train)
y_pred_train_linear = linear_model.predict(X_test)


In [19]:
#sigmoid function
def sigmoid(x):
  return 1/(1+np.exp(-x))

In [24]:
y_pred_sigmoid = sigmoid(y_pred_train_linear)
y_pred_binary = (y_pred_sigmoid >= 0.5).astype(int)

accuracy_linear_sigmoid = accuracy_score(Y_test, y_pred_binary)
print(f'Accuracy of Linear Regression with Sigmoid: {accuracy_linear_sigmoid:.4f}')


Accuracy of Linear Regression with Sigmoid: 0.5738


In [29]:
LogisticModel = LogisticRegression(max_iter=500, solver='saga')
LogisticModel.fit(X_train, Y_train)
y_pred_test_logistic = LogisticModel.predict(X_test)


accuracy_logistic = accuracy_score(Y_test, y_pred_test_logistic)
print(f'Accuracy of Logistic Regression: {accuracy_logistic:.4f}')

Accuracy of Logistic Regression: 0.8689


In [30]:
# Compare results
print(f'Accuracy Difference: {accuracy_logistic - accuracy_linear_sigmoid:.4f}')

Accuracy Difference: 0.2951


#Task 02

In [31]:
def binary_cross_entropy(y, y_pred):
  epsilon = 1e-15
  y_pred = np.clip(y_pred, epsilon, 1-epsilon)
  return -np.mean(y*np.log(y_pred) + (1-y)*np.log(1-y_pred))

In [32]:
bce_loss_linear = binary_cross_entropy(Y_test, y_pred_sigmoid)
print(f'Binary Cross-Entropy Loss (Linear Regression + Sigmoid): {bce_loss_linear:.4f}')

bce_loss_logistic = binary_cross_entropy(Y_test, LogisticModel.predict_proba(X_test)[:, 1])
print(f'Binary Cross-Entropy Loss (Logistic Regression): {bce_loss_logistic:.4f}')

Binary Cross-Entropy Loss (Linear Regression + Sigmoid): 0.6067
Binary Cross-Entropy Loss (Logistic Regression): 0.2897


#Task 03

In [33]:
pip install ucimlrepo

In [38]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
wine_quality = fetch_ucirepo(id=186)

# data (as pandas dataframes)
X = wine_quality.data.features
y = wine_quality.data.targets



In [39]:
# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
model = LinearRegression()

cv_scores = []


for fold, (train_index, val_index) in enumerate(kf.split(X_train, y_train)):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    model.fit(X_train_fold, y_train_fold)
    y_val_pred_fold = model.predict(X_val_fold)

    mse_fold = mean_squared_error(y_val_fold, y_val_pred_fold)
    cv_scores.append(mse_fold)
    print(f"Fold {fold + 1}: Mean Squared Error on Validation Set: {mse_fold}")

print(f'\nCross-validation scores: {cv_scores}')
print(f'Average MSE: {np.mean(cv_scores):.4f}')


y_test_pred = model.predict(X_test)
mse_test = mean_squared_error(y_test, y_test_pred)
print(f"\nMean Squared Error on Test Set: {mse_test}")


Fold 1: Mean Squared Error on Validation Set: 0.5506450248567712
Fold 2: Mean Squared Error on Validation Set: 0.6053534395174005
Fold 3: Mean Squared Error on Validation Set: 0.5527447033503022
Fold 4: Mean Squared Error on Validation Set: 0.5049041294861533
Fold 5: Mean Squared Error on Validation Set: 0.523273406168213

Cross-validation scores: [0.5506450248567712, 0.6053534395174005, 0.5527447033503022, 0.5049041294861533, 0.523273406168213]
Average MSE: 0.5474

Mean Squared Error on Test Set: 0.5336959214985676


#Task 04

In [59]:
def ForwardPropagation(X1, X2, w1, w2, b):
    z = (w1 * X1) + (w2 * X2) + b
    a = sigmoid(z)
    return a

In [60]:
def LossCalculation(a, y):
    loss = - (y * np.log(a) + (1 - y) * np.log(1 - a))
    return loss

In [61]:
def BackwardPropagation(X1, X2, a, y, w1, w2, b, learning_rate=0.1):
    dz = a - y  # Gradient of loss w.r.t. activation
    dw1 = dz * X1
    dw2 = dz * X2
    db = dz

    # Update weights
    w1 -= learning_rate * dw1
    w2 -= learning_rate * dw2
    b -= learning_rate * db

    return w1, w2, b

In [62]:
def MainLoop(X1, X2, y, w1, w2, b, loops=5):
    for i in range(loops):
        print(f'Iteration {i+1}:')
        a = ForwardPropagation(X1, X2, w1, w2, b)
        loss = LossCalculation(a, y)
        print(f'Prediction: {a:.4f}, Loss: {loss:.4f}')
        w1, w2, b = BackwardPropagation(X1, X2, a, y, w1, w2, b)
        print(f'Updated Weights -> w1: {w1:.4f}, w2: {w2:.4f}, b: {b:.4f}\n')

In [63]:
# Initialize variables
w1, w2, b = 0.5, 0.31, 0.09
X1, X2, y = 5, 3, 1

# Run training loop
MainLoop(X1, X2, y, w1, w2, b)

Iteration 1:
Prediction: 0.9713, Loss: 0.0292
Updated Weights -> w1: 0.5144, w2: 0.3186, b: 0.0929

Iteration 2:
Prediction: 0.9739, Loss: 0.0264
Updated Weights -> w1: 0.5274, w2: 0.3264, b: 0.0955

Iteration 3:
Prediction: 0.9762, Loss: 0.0241
Updated Weights -> w1: 0.5393, w2: 0.3336, b: 0.0979

Iteration 4:
Prediction: 0.9780, Loss: 0.0222
Updated Weights -> w1: 0.5503, w2: 0.3402, b: 0.1001

Iteration 5:
Prediction: 0.9796, Loss: 0.0206
Updated Weights -> w1: 0.5605, w2: 0.3463, b: 0.1021

